In [22]:
import cobra
import sys
sys.path.append('../../../utils/')
import importlib
import graph_utils 
importlib.reload(graph_utils)
import pickle
import pandas as pd
import numpy as np

###  Read Graph

In [23]:
with open('../../../Knowledge_graph/graph.pkl','rb') as handle:
    graph=pickle.load(handle)

### Read proteomic Data from Schmidt et al. (2016)

In [24]:
data=pd.read_csv('./Mori_et_al_2021_glucose_minimal_medium_mass_fractions.csv',index_col=0).set_index('uniprot_id')
data.head()

,bnum,A1-1,A1-2,A1-3,C1,F1-1,F1-2,F1-3
uniprot_id,,,,,,,,
P46482,b3241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
P46481,b3240,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
P67662,b3243,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
P46478,b3242,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
P31119,b2836,0.000044,0.000048,0.000046,0.000046,0.000048,0.000045,0.000046


### read proteomics metadata

In [25]:
metadata=pd.read_csv('./Mori_et_al_2021_parsed_metadata.csv')
metadata.head()

,condition_id,strain,type,condition,carbon_source,bigg_carbon_source,growth_rate
0,A1-1,EQ353,batch,aerobic,glucose,glc__D_e,0.69
1,A1-2,EQ353,batch,aerobic,glucose,glc__D_e,0.69
2,A1-3,EQ353,batch,aerobic,glucose,glc__D_e,0.69
3,C1,EQ353,batch,aerobic,glucose,glc__D_e,0.73
4,F1-1,EQ353,batch,aerobic,glucose,glc__D_e,0.71


Parametrise the polypeptide nodes in the graph

### Read PaxDB data

In [26]:
paxdb_data=pd.read_csv('../PAXDB/paxDB_integrated.tsv',sep='\t').set_index('#string_external_id')

In [27]:
conditions=metadata['condition_id'].unique().tolist()+['paxDB']

In [28]:
conditions


['A1-1', 'A1-2', 'A1-3', 'C1', 'F1-1', 'F1-2', 'F1-3', 'paxDB']

In [29]:
pp_nodes=[node for node in graph.nodes() if graph.nodes[node]['type']=='polypeptide']
pp_ids=data.index.tolist()
for pp in pp_nodes:
    if ('UNIPROT' in graph.nodes[pp]['annotation']):
        if (graph.nodes[pp]['annotation']['UNIPROT'] in pp_ids):
            pp_uniprot=graph.nodes[pp]['annotation']['UNIPROT']
            for condition in conditions:
                if condition in metadata['condition_id'].unique():
                    graph.nodes[pp][condition]=data.loc[pp_uniprot,condition]
                elif condition=='paxDB':
                    if 'bnum' in graph.nodes[pp]['gene']:
                        bnum=graph.nodes[pp]['gene']['bnum']
                        #Add PAXDB data
                        if bnum in paxdb_data.index:
                            graph.nodes[pp]['paxDB']=paxdb_data.loc[bnum,'abundance']

example_pp='ADENYL-KIN-MONOMER'
graph.nodes[example_pp]

{'id': 'ADENYL-KIN-MONOMER',
 'type': 'polypeptide',
 'biocyc_id': 'ECOLI:ADENYL-KIN-MONOMER',
 'mw': 23.586,
 'annotation': {'ALPHAFOLD': 'P69441',
  'PFAM': 'PF05191',
  'PDB': '4AKE',
  'INTERPRO': 'IPR006259',
  'SMR': 'P69441',
  'DIP': 'DIP-47903N',
  'PROSITE': 'PS00113',
  'PRIDE': 'P69441',
  'PANTHER': 'PTHR23359',
  'PRINTS': 'PR00094',
  'PRODB': 'PRO_000022061',
  'ECOLIWIKI': 'b0474',
  'MODBASE': 'P69441',
  'UNIPROT': 'P69441',
  'REFSEQ': 'NP_415007'},
 'gene': {'id': 'EG10032', 'bnum': 'b0474', 'name': 'adk'},
 'A1-1': 0.001201691,
 'A1-2': 0.001180291,
 'A1-3': 0.001232642,
 'C1': 0.00117967,
 'F1-1': 0.001246058,
 'F1-2': 0.001229338,
 'F1-3': 0.001253089,
 'paxDB': 2164.0}

### Compute node abundance recursively

In [30]:
def is_share_subunit(graph,node):
    subunit_edges=graph.in_edges(node,data=True)
    if len(subunit_edges)>1:
        return True
    else:
        return False


def get_node_abundance(graph,node,attribute,operation='min',add_attribute=False,rel_error_thresh=0):
    '''
    The abundance of node i is computed as f(a_j/w_ij for j in children(i))
    where f is a function of choice (here, max, min, avg)
    and a_j is the abundance of node j and w_ij is the weight of the edge between i and j
    '''

    if operation=='min':
        f=np.min
    elif operation=='max':
        f=np.max
    elif operation=='avg':
        f=np.mean

    
    if graph.nodes[node]['type']=='polypeptide':
        return graph.nodes[node][attribute] if attribute in graph.nodes[node].keys() else None
    elif graph.nodes[node]['type']=='spontaneous':
        return 0
    else:
        valid_subunit_edge_types=['subunit_composition','protein_modification']
        children=[child for child in graph.successors(node) if graph.edges[(node,child)]['type'] in valid_subunit_edge_types]
        children_is_shared_subunit=np.array([is_share_subunit(graph,child) for child in children])
        children_abundances=[get_node_abundance(graph,child,attribute) for child in children]
        if np.any([x is None for x in children_abundances]):
            return None
        elif np.max(children_abundances)==0:
            return 0
        else:
            children_weights=[graph.edges[(node,child)]['weight'] for child in children]
            normalised_abundances=np.array([a/w for a,w in zip(children_abundances,children_weights)])

            rel_variation=normalised_abundances[~children_is_shared_subunit]/np.max(normalised_abundances[~ children_is_shared_subunit])
            if np.any(rel_variation<=rel_error_thresh):
                print(f'Warning: relative variation of normalised children abundances for node {node} in {attribute} is less than threshold: {rel_error_thresh}. Discarding estimation for this node ')
                node_abundance= None
            else:
                node_abundance= f(normalised_abundances[~children_is_shared_subunit])
            
            if add_attribute:
                graph.nodes[node][attribute]=node_abundance
            return node_abundance

def recursive_abundance_computation(graph,attributes,operation='min',add_attribute=False):
    rxn_nodes=[node for node in graph.nodes if graph.nodes[node]['type']=='reaction']
    abundance_table={'enzyme':[],'reaction':[]}
    for attribute in attributes:
        abundance_table[attribute]=[]
    for rxn_node in rxn_nodes:
        #Look at all isoenzymes catalyzing the reactio
        catalysis_edges=['catalysis','secondary_catalysis']
        children=[child for child in graph.successors(rxn_node) if graph.edges[(rxn_node,child)]['type'] in catalysis_edges]
        for child in children:
            #Get the abundance of the child
            abundance_table['enzyme'].append(child)
            abundance_table['reaction'].append(rxn_node)
            for attribute in attributes:
                child_abundance=get_node_abundance(graph,child,attribute,operation=operation,add_attribute=add_attribute)
                abundance_table[attribute].append(child_abundance)
            else:
                pass
    abundance_df=pd.DataFrame.from_dict(abundance_table)
    return abundance_df


In [31]:
abundance_df=recursive_abundance_computation(graph,
                                             attributes=conditions,
                                             operation='max')

In [32]:
abundance_df

,enzyme,reaction,A1-1,A1-2,A1-3,C1,F1-1,F1-2,F1-3,paxDB
0,ADENYL-KIN-MONOMER,bigg:NDPK5,0.001202,0.001180,0.001233,0.001180,0.001246,0.001229,0.001253,2164.000
1,NUCLEOSIDE-DIP-KIN-CPLX,bigg:NDPK5,0.000289,0.000294,0.000279,0.000242,0.000276,0.000262,0.000265,526.500
2,AROE-MONOMER,bigg:SHK3Dr,0.000048,0.000043,0.000042,0.000052,0.000050,0.000048,0.000050,42.800
3,EG11234-MONOMER,bigg:SHK3Dr,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.464
4,NUCLEOSIDE-DIP-KIN-CPLX,bigg:NDPK6,0.000289,0.000294,0.000279,0.000242,0.000276,0.000262,0.000265,526.500
...,...,...,...,...,...,...,...,...,...,...
526,PITB-MONOMER,bigg:PIt2rpp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
527,EG11919-MONOMER,bigg:PItpp,0.000022,0.000023,0.000024,0.000031,0.000036,0.000019,0.000000,3.420
528,YCHM-MONOMER,bigg:SUCCt1pp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.200
529,EG11512-MONOMER,bigg:SUCCt1pp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [33]:
for operation in ['min','max','avg']:
    abundance_df=recursive_abundance_computation(graph,
                                                      attributes=conditions,
                                                      operation=operation)
    abundance_df=abundance_df.drop('reaction',axis='columns').drop_duplicates()
    #total_measured_proteome=0.828118444476821 #g/gDW taken from schmidt 2016, need to find the right value for each condition in Mori et al 2021
    for condition in conditions:
        abundance_df[condition]=abundance_df[condition]#total_measured_proteome
    abundance_df.to_csv(f'./model_mapped_abundances/Mori_et_al_2021_mapped_mass_fractions_{operation}.csv',index=False)

In [34]:
mass_abundance_df.sum()

A1-1_g_gDW     1.130946e-07
A1-2_g_gDW     1.134397e-07
A1-3_g_gDW     1.153634e-07
C1_g_gDW       1.136241e-07
F1-1_g_gDW     1.126552e-07
F1-2_g_gDW     1.132532e-07
F1-3_g_gDW     1.129292e-07
paxDB_g_gDW    6.348752e-02
dtype: float64